**1) Build Article Embeddings**

In [8]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
from sklearn.decomposition import PCA
from numpy.linalg import norm
import json



def filter_users_with_min_clicks(df, min_clicks=5):
    df['click_count'] = df['history'].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)
    return df[df['click_count'] >= min_clicks].drop(columns=['click_count'])


# ----------------------------------------------------
# Step 1: Load and Preprocess Article Embeddings
# ----------------------------------------------------
def parse_entity_list(entity_str):
    try:
        data = json.loads(entity_str)
        return [obj['WikidataId'] for obj in data if 'WikidataId' in obj]
    except (json.JSONDecodeError, TypeError):
        return []

def build_article_embeddings(news_df, entity_embeddings, use_abstract=True):
    article_vectors = {}
    for news_id, row in news_df.iterrows():
        title_ids = parse_entity_list(row['title_entities'])
        abstract_ids = parse_entity_list(row['abstract_entities']) if use_abstract else []
        entity_ids = title_ids + abstract_ids
        vectors = [entity_embeddings[eid] for eid in entity_ids if eid in entity_embeddings]
        if vectors:
            article_vectors[news_id] = np.mean(vectors, axis=0)
    return article_vectors

# ----------------------------------------------------
# Step 2: Fit PCA on Article Embeddings
# ----------------------------------------------------
def apply_pca_to_article_vectors(article_vectors, n_components=50):
    article_ids = list(article_vectors.keys())
    matrix = np.stack([article_vectors[aid] for aid in article_ids])
    pca = PCA(n_components=n_components)
    reduced_matrix = pca.fit_transform(matrix)
    reduced_article_vectors = {aid: reduced_matrix[i] for i, aid in enumerate(article_ids)}
    return reduced_article_vectors, pca

# ----------------------------------------------------
# Step 3: Build User Profiles (from article embeddings)
# ----------------------------------------------------
def build_user_profiles_embedding_based(behaviors_df, article_vectors):
    user_profiles = {}
    for _, row in behaviors_df.iterrows():
        user_id = row['user_id']
        if not isinstance(row['history'], str):
            continue
        clicked_ids = row['history'].split()
        vectors = [article_vectors[aid] for aid in clicked_ids if aid in article_vectors]
        if vectors:
            user_profiles[user_id] = np.mean(vectors, axis=0)
    return user_profiles

# ----------------------------------------------------
# Step 4: Recommender Function Using PCA
# ----------------------------------------------------
def recommend_for_user(user_id, user_profiles, cluster_to_articles, train_behaviors_df,
                       article_vectors, pca, top_clusters=2, top_k=5):
    if user_id not in user_profiles:
        return []

    distribution = user_profiles[user_id]
    cluster_indices = np.argsort(distribution)[::-1]
    selected_clusters = cluster_indices[:top_clusters]

    candidate_articles = list({aid for c in selected_clusters for aid in cluster_to_articles[c]})

    full_clicked = []
    user_rows = train_behaviors_df[train_behaviors_df['user_id'] == user_id]
    for _, row in user_rows.iterrows():
        if isinstance(row['history'], str):
            full_clicked.extend(row['history'].split())

    valid_embeddings = [article_vectors[aid] for aid in full_clicked if aid in article_vectors]
    if not valid_embeddings:
        return candidate_articles[:top_k]

    user_embedding = np.mean(valid_embeddings, axis=0)
    user_embedding_reduced = user_embedding  # already reduced


    scored = []
    for aid in candidate_articles:
        if aid not in article_vectors:
            continue
        art_vec = article_vectors[aid]
        sim = np.dot(user_embedding_reduced, art_vec) / (norm(user_embedding_reduced) * norm(art_vec))
        scored.append((aid, sim))

    scored.sort(key=lambda x: x[1], reverse=True)
    return [x[0] for x in scored[:top_k]]

# ----------------------------------------------------
# Step 5: Evaluation Functions
# ----------------------------------------------------
def parse_impressions(impressions_str):
    items = impressions_str.split()
    all_ids = []
    clicked = []
    for x in items:
        article_id, label_str = x.split('-')
        all_ids.append(article_id)
        if label_str == '1':
            clicked.append(article_id)
    return all_ids, clicked

"""def dcg_at_k(recommended, actual_clicked, K=5):
    return sum(1 / np.log2(i + 2) for i, item in enumerate(recommended[:K]) if item in actual_clicked)

def idcg_at_k(actual_clicked, K=5):
    num_relevant = min(len(actual_clicked), K)
    return sum(1 / np.log2(i + 2) for i in range(num_relevant)) or 1

def ndcg_at_k(recommended, actual_clicked, K=5):
    return dcg_at_k(recommended, actual_clicked, K) / idcg_at_k(actual_clicked, K)

def auc_at_k(recommended, actual_clicked, K=5):
    recommended = recommended[:K]
    relevance = [1 if item in actual_clicked else 0 for item in recommended]
    num_pos = sum(relevance)
    num_neg = len(relevance) - num_pos
    if num_pos == 0: return 0.0
    if num_neg == 0: return 1.0
    correct_pairs = sum(1 for i in range(len(relevance)) if relevance[i] == 1
                        for j in range(i+1, len(relevance)) if relevance[j] == 0)
    return correct_pairs / (num_pos * num_neg)

def mrr_at_k(recommended, actual_clicked, K=5):
    recommended = recommended[:K]
    for i, item in enumerate(recommended):
        if item in actual_clicked:
            return 1.0 / (i + 1)
    return 0.0

def evaluate_model_self(user_profiles, cluster_to_articles, train_behaviors_df,
                   test_behaviors_df, article_vectors, pca, K=5):
    ndcg_scores, mrr_scores, auc_scores = [], [], []
    for _, row in test_behaviors_df.iterrows():
        user_id = row['user_id']
        if pd.isna(row['impressions']):
            continue
        all_ids, clicked = parse_impressions(row['impressions'])
        if not clicked:
            continue

        recommended = recommend_for_user(user_id, user_profiles, cluster_to_articles,
                                         train_behaviors_df, article_vectors, pca,
                                         top_clusters=5, top_k=len(all_ids))
        ndcg_scores.append(ndcg_at_k(recommended, clicked, K))
        mrr_scores.append(mrr_at_k(recommended, clicked, K))
        auc_scores.append(auc_at_k(recommended, clicked, K))

    return np.mean(ndcg_scores), np.mean(auc_scores), np.mean(mrr_scores)

    """



"def dcg_at_k(recommended, actual_clicked, K=5):\n    return sum(1 / np.log2(i + 2) for i, item in enumerate(recommended[:K]) if item in actual_clicked)\n\ndef idcg_at_k(actual_clicked, K=5):\n    num_relevant = min(len(actual_clicked), K)\n    return sum(1 / np.log2(i + 2) for i in range(num_relevant)) or 1\n\ndef ndcg_at_k(recommended, actual_clicked, K=5):\n    return dcg_at_k(recommended, actual_clicked, K) / idcg_at_k(actual_clicked, K)\n\ndef auc_at_k(recommended, actual_clicked, K=5):\n    recommended = recommended[:K]\n    relevance = [1 if item in actual_clicked else 0 for item in recommended]\n    num_pos = sum(relevance)\n    num_neg = len(relevance) - num_pos\n    if num_pos == 0: return 0.0\n    if num_neg == 0: return 1.0\n    correct_pairs = sum(1 for i in range(len(relevance)) if relevance[i] == 1\n                        for j in range(i+1, len(relevance)) if relevance[j] == 0)\n    return correct_pairs / (num_pos * num_neg)\n\ndef mrr_at_k(recommended, actual_clicked,

In [9]:
import os

# ----- File Paths -----
dataset_dir = "../MINDsmall_train"
news_file = os.path.join(dataset_dir, "news.tsv")
embedding_file = os.path.join(dataset_dir, "entity_embedding.vec")
train_behaviors_file = os.path.join(dataset_dir, "behaviors.tsv")
test_behaviors_file = "../MINDsmall_dev/behaviors.tsv"

# ----- Load data -----
news_cols = ['news_id', 'category', 'subcategory', 'title', 'abstract', 'url', 'title_entities', 'abstract_entities']
news_df = pd.read_csv(news_file, sep='\t', header=None, names=news_cols)
news_df.set_index('news_id', inplace=True)

train_behaviors_df = pd.read_csv(train_behaviors_file, sep='\t', header=None,
    names=['impression_id', 'user_id', 'time', 'history', 'impressions'])

train_behaviors_df = filter_users_with_min_clicks(train_behaviors_df, min_clicks=5)

test_behaviors_df = pd.read_csv(test_behaviors_file, sep='\t', header=None,
    names=['impression_id', 'user_id', 'time', 'history', 'impressions'])

test_behaviors_df = filter_users_with_min_clicks(test_behaviors_df, min_clicks=5)

# Filter test behaviors to only include users present in train behaviors
valid_users = set(train_behaviors_df['user_id'].unique())
test_behaviors_df = test_behaviors_df[test_behaviors_df['user_id'].isin(valid_users)]


# ----- Load entity embeddings -----
entity_embeddings = {}
with open(embedding_file, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split()
        entity_id = parts[0]
        vector = np.array([float(x) for x in parts[1:]], dtype=np.float32)
        entity_embeddings[entity_id] = vector

# ----- Step 1: Build article embeddings -----
article_vectors = build_article_embeddings(news_df, entity_embeddings, use_abstract=True)

# ----- Step 2: Apply PCA -----
reduced_article_vectors, pca = apply_pca_to_article_vectors(article_vectors, n_components=50)

# ----- Step 3: Cluster articles (e.g. using KMeans) -----
from sklearn.cluster import KMeans
X = np.stack(list(reduced_article_vectors.values()))
article_ids = list(reduced_article_vectors.keys())
kmeans = KMeans(n_clusters=20, random_state=42, n_init=10)
kmeans.fit(X)
cluster_assignments = {aid: int(label) for aid, label in zip(article_ids, kmeans.labels_)}
cluster_to_articles = defaultdict(list)
for aid, label in cluster_assignments.items():
    cluster_to_articles[label].append(aid)

# ----- Step 4: Build user cluster profiles -----
def build_cluster_distribution_profiles(behaviors_df, cluster_assignments, num_clusters=50):
    user_profiles = defaultdict(lambda: np.zeros(num_clusters, dtype=np.float32))
    for _, row in behaviors_df.iterrows():
        if not isinstance(row['history'], str):
            continue
        for aid in row['history'].split():
            if aid in cluster_assignments:
                user_profiles[row['user_id']][cluster_assignments[aid]] += 1
    return {u: v / (v.sum() or 1.0) for u, v in user_profiles.items()}

user_profiles = build_cluster_distribution_profiles(train_behaviors_df, cluster_assignments)
"""
# ----- Step 5: Evaluate -----
avg_ndcg, avg_auc, avg_mrr = evaluate_model_self(
    user_profiles=user_profiles,
    cluster_to_articles=cluster_to_articles,
    train_behaviors_df=train_behaviors_df,
    test_behaviors_df=test_behaviors_df,
    article_vectors=reduced_article_vectors,
    pca=pca,
    K=5
)

# ----- Print Results -----
print(f"Evaluation Results:")
print(f"NDCG@5: {avg_ndcg:.4f}")
print(f"AUC@5:  {avg_auc:.4f}")
print(f"MRR@5:  {avg_mrr:.4f}")
"""

'\n# ----- Step 5: Evaluate -----\navg_ndcg, avg_auc, avg_mrr = evaluate_model_self(\n    user_profiles=user_profiles,\n    cluster_to_articles=cluster_to_articles,\n    train_behaviors_df=train_behaviors_df,\n    test_behaviors_df=test_behaviors_df,\n    article_vectors=reduced_article_vectors,\n    pca=pca,\n    K=5\n)\n\n# ----- Print Results -----\nprint(f"Evaluation Results:")\nprint(f"NDCG@5: {avg_ndcg:.4f}")\nprint(f"AUC@5:  {avg_auc:.4f}")\nprint(f"MRR@5:  {avg_mrr:.4f}")\n'

In [10]:
import sys
sys.path.append("..")
from utils.evaluation import evaluate_model

class ClusteredContentRecommender:
    def __init__(self, user_profiles, cluster_to_articles, train_behaviors_df, article_vectors, pca):
        self.user_profiles = user_profiles
        self.cluster_to_articles = cluster_to_articles
        self.train_behaviors_df = train_behaviors_df
        self.article_vectors = article_vectors
        self.pca = pca

    def recommend(self, user_id, N=5):
        return recommend_for_user(
            user_id=user_id,
            user_profiles=self.user_profiles,
            cluster_to_articles=self.cluster_to_articles,
            train_behaviors_df=self.train_behaviors_df,
            article_vectors=self.article_vectors,
            pca=self.pca,
            top_clusters=2,
            top_k=N
        )

# Instantiate the recommender
recommender = ClusteredContentRecommender(
    user_profiles=user_profiles,
    cluster_to_articles=cluster_to_articles,
    train_behaviors_df=train_behaviors_df,
    article_vectors=reduced_article_vectors,
    pca=pca
)

# Evaluate the model
ndcg, auc, mrr = evaluate_model(recommender, test_behaviors_df, K=5)

print("\n **Evaluation Results:**")
print(f" NDCG@5: {ndcg:.4f}")
print(f" AUC@5:  {auc:.4f}")
print(f" MRR@5:  {mrr:.4f}")




 **Evaluation Results:**
 NDCG@5: 0.0003
 AUC@5:  0.0003
 MRR@5:  0.0004
